In [1]:
#!pip install nltk
#nltk.download()
#!pip install gensim
#import sqlite3
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('punkt')
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer , PorterStemmer
from textblob import TextBlob, Word
from nltk.stem import PorterStemmer as ps
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
import numpy as np
import scipy.sparse as sp
import gensim
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from nltk import ne_chunk
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
import json
import pickle
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you?',
    'Is anyone there?',
    'Hello',
    'Good day',
    "What's up",
    'how are ya',
    'heyy',
    'whatsup',
    '??? ??? ??'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['cya',
    'see you',
    'bye bye',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Bye',
    'Have a Good day',
    'talk to you later',
    'ttyl',
    'i got to go',
    'gtg'],
   'responses': ['Sad to see you go :(',
    'Talk to you later',
    'Goodbye!',
    'Come back soon'],
   'context_set': ''},
  {'tag': 'creator',
   'patterns': ['what is the name of your developers',
    'what is the name of your creators',
    'what is the name of the developers',
    'what is the name of the creators',
    'who created you',
    'your developers',
    'your creators',
    'who are your developers',
    'developers',
  

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        # add each elements into list
        words.extend(w)
        #combination between patterns and intents
        documents.append((w, intent['tag']))#add single element into end of list
        # add to tag in our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
# lemmatize, lower each word and remove duplicates
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents\n", documents, "\n")
# classes = intents[tag]
print (len(classes), "classes\n", classes, "\n")
# words = all words, vocabulary
print (len(words), "unique lemmatized words\n", words, "\n")
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

405 documents
 [(['Hi'], 'greeting'), (['How', 'are', 'you', '?'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['What', "'s", 'up'], 'greeting'), (['how', 'are', 'ya'], 'greeting'), (['heyy'], 'greeting'), (['whatsup'], 'greeting'), (['?', '?', '?', '?', '?', '?', '?', '?'], 'greeting'), (['cya'], 'goodbye'), (['see', 'you'], 'goodbye'), (['bye', 'bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['Bye'], 'goodbye'), (['Have', 'a', 'Good', 'day'], 'goodbye'), (['talk', 'to', 'you', 'later'], 'goodbye'), (['ttyl'], 'goodbye'), (['i', 'got', 'to', 'go'], 'goodbye'), (['gtg'], 'goodbye'), (['what', 'is', 'the', 'name', 'of', 'your', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'your', 'creators'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'creato

In [5]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words
    pattern_words = doc[0]
    # convert pattern_words in lower case
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create bag of words array,if word match found in current pattern then put 1 otherwise 0.[row * colm(263)]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # in output array 0 value for each tag ang 1 value for matched tag.[row * colm(8)]
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle training and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test. X - patterns(words), Y - intents(tags)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\ahmed\AppData\Local\Temp\ipykernel_15972\677733079.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [6]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("*"*50)
print("model created")

Epoch 1/200
81/81 [==============================] - 2s 5ms/step - loss: 3.6063 - accuracy: 0.0395
Epoch 2/200
81/81 [==============================] - 0s 4ms/step - loss: 3.4704 - accuracy: 0.1432
Epoch 3/200
81/81 [==============================] - 1s 8ms/step - loss: 3.2516 - accuracy: 0.2123
Epoch 4/200
81/81 [==============================] - 0s 4ms/step - loss: 3.0058 - accuracy: 0.2198
Epoch 5/200
81/81 [==============================] - 0s 4ms/step - loss: 2.7083 - accuracy: 0.3037
Epoch 6/200
81/81 [==============================] - 0s 4ms/step - loss: 2.5022 - accuracy: 0.3432
Epoch 7/200
81/81 [==============================] - 0s 4ms/step - loss: 2.2371 - accuracy: 0.4519
Epoch 8/200
81/81 [==============================] - 0s 5ms/step - loss: 1.9892 - accuracy: 0.5037
Epoch 9/200
81/81 [==============================] - 0s 4ms/step - loss: 1.8622 - accuracy: 0.5284
Epoch 10/200
81/81 [==============================] - 0s 4ms/step - loss: 1.6832 - accuracy: 0.5679
Epoch 11/

81/81 [==============================] - 0s 5ms/step - loss: 0.1354 - accuracy: 0.9556
Epoch 84/200
81/81 [==============================] - 0s 4ms/step - loss: 0.1355 - accuracy: 0.9556
Epoch 85/200
81/81 [==============================] - 0s 4ms/step - loss: 0.1573 - accuracy: 0.9531
Epoch 86/200
81/81 [==============================] - 0s 5ms/step - loss: 0.1196 - accuracy: 0.9630
Epoch 87/200
81/81 [==============================] - 0s 5ms/step - loss: 0.1332 - accuracy: 0.9605
Epoch 88/200
81/81 [==============================] - 0s 5ms/step - loss: 0.1101 - accuracy: 0.9506
Epoch 89/200
81/81 [==============================] - 0s 5ms/step - loss: 0.1346 - accuracy: 0.9556
Epoch 90/200
81/81 [==============================] - 0s 5ms/step - loss: 0.1396 - accuracy: 0.9506
Epoch 91/200
81/81 [==============================] - 0s 4ms/step - loss: 0.1370 - accuracy: 0.9580
Epoch 92/200
81/81 [==============================] - 0s 4ms/step - loss: 0.1071 - accuracy: 0.9630
Epoch 93/200


81/81 [==============================] - 0s 4ms/step - loss: 0.0772 - accuracy: 0.9679
Epoch 165/200
81/81 [==============================] - 0s 4ms/step - loss: 0.0665 - accuracy: 0.9802
Epoch 166/200
81/81 [==============================] - 0s 4ms/step - loss: 0.0598 - accuracy: 0.9827
Epoch 167/200
81/81 [==============================] - 0s 4ms/step - loss: 0.0732 - accuracy: 0.9654
Epoch 168/200
81/81 [==============================] - 0s 5ms/step - loss: 0.0515 - accuracy: 0.9827
Epoch 169/200
81/81 [==============================] - 0s 4ms/step - loss: 0.0973 - accuracy: 0.9654
Epoch 170/200
81/81 [==============================] - 0s 4ms/step - loss: 0.0651 - accuracy: 0.9753
Epoch 171/200
81/81 [==============================] - 0s 5ms/step - loss: 0.0678 - accuracy: 0.9852
Epoch 172/200
81/81 [==============================] - 1s 7ms/step - loss: 0.0549 - accuracy: 0.9827
Epoch 173/200
81/81 [==============================] - 0s 5ms/step - loss: 0.0482 - accuracy: 0.9852
Epoc

In [7]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    #print(sentence_words)
    # stem each word - create short form for word

    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    #print(sentence_words)

    return sentence_words

In [8]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern

    sentence_words = clean_up_sentence(sentence)
    #print(sentence_words)

    # bag of words - matrix of N words, vocabulary matrix

    bag = [0]*len(words) 
    #print(bag)

    for s in sentence_words:  
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
                #print ("found in bag: %s" % w)
    #print(bag)
    return(np.array(bag))

In [9]:
def predict_class(sentence, model):
    # filter out predictions below a threshold

    p = bow(sentence, words,show_details=False)
    #print(p)

    res = model.predict(np.array([p]))[0]
    #print(res)

    ERROR_THRESHOLD = 0.25

    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    #print(results)
    # sort by strength of probability

    results.sort(key=lambda x: x[1], reverse=True)
    #print(results)

    return_list = []

    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

    return return_list
    #print(return_list)

In [10]:
def getResponse(ints, intents_json):

    tag = ints[0]['intent']
    #print(tag)

    list_of_intents = intents_json['intents']
    #print(list_of_intents)

    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [11]:
def chatbot_response(text): 
    ints = predict_class(text, model)

    #print(ints)

    res = getResponse(ints, intents)
    #print(res)
    return res

In [12]:
def start_chat():
    print("Bot : This is Siri! Your Personal Assistant.\n\n")
    while True:
        inp = str(input()).lower()
        if inp.lower()=="end":
            break
        if inp.lower()== '' or inp.lower()== '*':
            print('please re-phrase your query!')
            print('-'*50)
        else:
            print(f"Bot: {chatbot_response(inp)}"+'\n')
            print("-"*50)

In [15]:
#start_chat()

In [16]:
import random
import tkinter as tk
from tkinter import *

root=tk.Tk()
filename="Chat Bot"
root.title(f"Chat Bot")
root.geometry('500x400')
root.resizable(False, False)
message=tk.StringVar()

chat_win=Frame(root,bd=1,bg='white',width=50,height=8)
chat_win.place(x=6,y=6,height=300,width=488)

textcon=tk.Text(chat_win,bd=1,bg='white',width=50,height=8)
textcon.pack(fill="both",expand=True)

mes_win=Entry(root,width=30,xscrollcommand=True,textvariable=message)
mes_win.place(x=6,y=310,height=60,width=380)
mes_win.focus()

textcon.config(fg='black')
textcon.tag_config('usr',foreground='black')
textcon.insert(END,"Bot: This is Siri! Your Personal Assistant.\n\n")
mssg=mes_win.get()

exit_list = ['exit','break','quit','see you later','chat with you later','end the chat','bye','ok bye']

def greet_res(text):
    text=text.lower()
    bot_greet=['hi','hello','hola','hey','howdy']
    usr_greet=['hi','hey','hello','hola','greetings','wassup','whats up']
    for word in text.split():
        if word in usr_greet:
            return random.choice(bot_greet)

def send_msz(event=None):
    usr_input = message.get()
    usr_input = usr_input.lower()
    textcon.insert(END, f'You: {usr_input}'+'\n','usr')
    if usr_input in exit_list:
        textcon.config(fg='black')
        textcon.insert(END,"Bot: Ok bye! Chat with you later\n")
        return root.destroy()
    else:
        textcon.config(fg='black')
        if greet_res(usr_input) != None:
            lab=f"Bot: {greet_res(usr_input)}"+'\n'
            textcon.insert(END,lab)
            mes_win.delete(0,END)
        else:
            lab = f"Bot: {chatbot_response(usr_input)}"+'\n'
            textcon.insert(END,lab)
            mes_win.delete(0,END)

button_send=Button(root,text='Send',bg='dark green',activebackground='grey',command=send_msz,width=12,height=5,font=('Arial'))
button_send.place(x=376,y=310,height=60,width=110)
root.bind('<Return>', send_msz,button_send)
root.mainloop()

1/1 [==============================] - 0s 58ms/step
